In [1]:
import pandas as pd
import numpy as np
from store_sales import encode_features, get_mae_score_cross_validation
from sklearn.model_selection import TimeSeriesSplit
import xgboost as xgb

In [2]:
data = pd.read_csv('../data/prepared_data/prepared_data.csv')
data.head()

,date,store_number,item_family,items_on_promotion,item_sales,city,state,store_type,store_cluster,oil_price,...,is_holiday_transferred,is_during_oil_prices_falling,is_popular_store,is_popular_cluster,is_special_non_working_day,is_national_holiday,is_state_pichincha,is_city_quito_or_cayambe,is_store_type_A,number_of_days_since_earthquake
0,2013-01-01,1,AUTOMOTIVE,0,0.0,Quito,Pichincha,D,13,93.14,...,False,0,0,0,0,1,1,1,0,-1201
1,2013-01-01,1,BABY CARE,0,0.0,Quito,Pichincha,D,13,93.14,...,False,0,0,0,0,1,1,1,0,-1201
2,2013-01-01,1,BEAUTY,0,0.0,Quito,Pichincha,D,13,93.14,...,False,0,0,0,0,1,1,1,0,-1201
3,2013-01-01,1,BEVERAGES,0,0.0,Quito,Pichincha,D,13,93.14,...,False,0,0,0,0,1,1,1,0,-1201
4,2013-01-01,1,BOOKS,0,0.0,Quito,Pichincha,D,13,93.14,...,False,0,0,0,0,1,1,1,0,-1201


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 24 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   date                             object 
 1   store_number                     int64  
 2   item_family                      object 
 3   items_on_promotion               int64  
 4   item_sales                       float64
 5   city                             object 
 6   state                            object 
 7   store_type                       object 
 8   store_cluster                    int64  
 9   oil_price                        float64
 10  day_type                         object 
 11  holiday_status                   object 
 12  holiday_location                 object 
 13  holiday_description              object 
 14  is_holiday_transferred           bool   
 15  is_during_oil_prices_falling     int64  
 16  is_popular_store                 int64  
 17  is_popul

In [4]:
split_ratio = 0.9
split_index = int(len(data) * split_ratio)
train_data = data.iloc[:split_index]
test_data = data.iloc[split_index:]

In [5]:
train_data, test_data = encode_features(train_data, test_data)

/Users/stanislavyatsevich/Desktop/Store_Sales/src/store_sales/functions.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['date'] = pd.to_datetime(train_data['date'])
/Users/stanislavyatsevich/Desktop/Store_Sales/src/store_sales/functions.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['days_since_start'] = (pd.to_datetime(train_data['date']) - pd.to_datetime('2013-01-01')).dt.days
/Users/stanislavyatsevich/Desktop/Store_Sales/src/store_sales/functions.py:60: SettingWithCopyWar

In [6]:
tscv = TimeSeriesSplit(n_splits=5)
xgboost = xgb.XGBRegressor(max_depth=2, n_jobs=-1, random_state=42)
#models = [xgboost]
#models_names = ['XGBoost']

#all_sales = pd.DataFrame()
mae_scores = []
#mape_scores = []

for store_num in train_data['store_number'].unique():
    for item_family in train_data['item_family'].unique():
        data = train_data[(train_data['store_number'] == store_num) & (train_data['item_family'] == item_family)]
        X = data.drop(['item_sales'], axis=1)
        y = data['item_sales']
        for train_index, test_index in tscv.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            mae_score = get_mae_score_cross_validation(X_train, X_test, y_train, y_test, xgboost)
            mae_scores.append(mae_score)

print('Average MAE for XGBoost:', np.array(mae_scores).mean())

Average MAE for XGBoost: 114.99605499438833


In [9]:
train_data['item_sales'].mean()

343.7749861442594